# Basic Scenario

In this basic scenario, we will be simulating a small data center equipped with
photovoltaic (PV) modules and a simple battery system. First, we need to import
all necessary vessim modules and datasets, we will be using for this scenario.

In [1]:
from vessim.actor import ComputingSystem, Generator
from vessim.controller import Monitor
from vessim.cosim import Environment
from vessim.power_meter import MockPowerMeter
from vessim.signal import HistoricalSignal
from vessim.storage import SimpleBattery

# Jupyter async bug fix
import nest_asyncio
nest_asyncio.apply()

The vessim Environment holds all simulation components and is defined at the
very start of our scenario. We initialize it with the start date of our
simulation. 

In [2]:
environment = Environment(sim_start="2020-06-11 00:00:00")

Central to our vessim Environment is the Microgrid component. The Microgrid
holds and connects all Actors, Controllers and Storage entities. Actors can
consume and produce power. An excess or deficiency in power can be fed to or
drawn from Storage. Controller entities can react to the Actors activity and the
total power delta that is drawn from or fed to the electrical grid. For now, the
only controller is a monitor, which stores all simulation data in a CSV file.

In [3]:
monitor = Monitor()  # stores simulation result on each step
environment.add_microgrid(
    actors=[
        ComputingSystem(power_meters=[MockPowerMeter(p=3), MockPowerMeter(p=7)]),
        Generator(signal=HistoricalSignal.from_dataset("solcast2022_global")),
    ],
    controllers=[monitor],
    storage=SimpleBattery(capacity=100),
    step_size=60,  # global step size (can be overridden by actors or controllers)
)

environment.run(until=24 * 3600) # 24h
monitor.to_csv("result.csv")

Required data files not present locally. Try downloading...


2024-02-16 16:23:59.861 | INFO     | mosaik.scenario:start:280 - Starting "Grid" as "Grid-0" ...
2024-02-16 16:23:59.862 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-0" ...
2024-02-16 16:23:59.863 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-1" ...
2024-02-16 16:23:59.863 | INFO     | mosaik.scenario:start:280 - Starting "Controller" as "Controller-0" ...
2024-02-16 16:23:59.865 | INFO     | mosaik.scenario:run:598 - Starting simulation.


Successfully downloaded and unpacked data files.


  0%|          | 16/86400 [00:00<00:06, 13443.28steps/s]


ValueError: Column needs to be specified.

This scenario includes two Actors: a computing system with two power consuming
nodes, and a generator which produces power based on historic solar data,
turning it into a PV module. 

All Actors must be subclassed from the `Actor` ABC from 
[vessim.actor](https://github.com/dos-group/vessim/blob/main/vessim/actor.py)
and at least implement the abstract method `p()` which returns the current power
consumption or production.

```python
class Actor(ABC):
    """Abstract base class representing a power consumer or producer."""

    def __init__(self, name: str, step_size: Optional[int] = None):
        self.name = name
        self.step_size = step_size

    @abstractmethod
    def p(self, now: datetime) -> float:
        """Current power consumption/production to be used in the grid simulation."""

    def state(self, now: datetime) -> dict:
        """Current state of the actor to be used in controllers."""
        return {}

    def finalize(self) -> None:
        """Perform any finalization tasks for the consumer.

        This method can be overridden by subclasses to implement necessary
        finalization steps.
        """
        return
```